In [1]:
import pandas as pd
import numpy as np

import warnings;
warnings.simplefilter('ignore')

In [2]:
# Ventas EPC
# ==============================================
df_ventas_epc = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                            sheet_name='6.Ventas EPC REAL')
col_conserved = []

for i, col in enumerate(df_ventas_epc.columns):
    if i == 0 and df_ventas_epc[col].dropna().empty:
        continue
    elif df_ventas_epc[col].dropna().empty:
        break
    else:
        col_conserved.append(col)
col_conserved = col_conserved[:-1]

df_ventas_epc = df_ventas_epc[df_ventas_epc[['Unnamed: 4']].dropna().index[0]:df_ventas_epc[['Unnamed: 4']].dropna().index[-1]][col_conserved]
cols_range_dates = pd.date_range(start="2025-01", end="2025-12", freq="MS").strftime("%Y-%m").tolist()
cols_strings = ['Rubro','Producto','Budget','Partida']
df_ventas_epc.columns = cols_strings+cols_range_dates 
df_ventas_epc[cols_strings] = df_ventas_epc[cols_strings].ffill()
df_ventas_epc[cols_range_dates] = df_ventas_epc[cols_range_dates].fillna(0)
df_ventas_epc['Nodo'] = np.where(df_ventas_epc['Partida'] == 'Ventas', 'Ingresos', 'Costos')
df_ventas_epc['Operación'] = np.where(df_ventas_epc['Partida'] == 'Ventas', 'I', 'E')
df_ventas_epc['Partida'] = df_ventas_epc['Partida'] + ' EPC'
df_epc_concat = df_ventas_epc.copy()

In [3]:
# Ingreso venta de energia
# ==============================================
df_ventas_energia = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='7,Ingreso venta de energia')

df_ventas_energia = df_ventas_energia[df_ventas_energia[['Unnamed: 0']].dropna().index[0]:df_ventas_energia[['Unnamed: 4']].dropna().index[-1]]
cols_strings = ['Producto','Budget']
df_ventas_energia.columns = cols_strings+cols_range_dates 
df_ventas_energia[cols_range_dates] = df_ventas_energia[cols_range_dates].fillna(0)
df_ventas_energia['Rubro'] = 'Energía'
df_ventas_energia['Nodo'] = 'Ingresos'
df_ventas_energia['Partida'] = 'Ingreso por venta de energía'
df_ventas_energia['Operación'] = 'I'

# Costos por venta de energía
# ==============================================
df_costos_energia = pd.DataFrame([[0,0,0,
                                    5757561,
                                    2927571,
                                    4166667,
                                    4166667,
                                    4166667,
                                    4166667,
                                    4166667,
                                    4166667,
                                    4166667]]
, columns= cols_range_dates)
df_costos_energia['Budget'] = df_costos_energia.sum(axis = 1)
df_costos_energia['Rubro'] = 'Energía'
df_costos_energia['Partida'] = 'Costos por venta de energía'
df_costos_energia['Producto'] = 'Energía'
df_costos_energia['Nodo'] = 'Costos'
df_costos_energia['Operación'] = 'E'

df_energia_concat = pd.concat([df_ventas_energia, df_costos_energia], axis = 0)


In [4]:
# Ingreso granjas
# ==============================================
df_granjas = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='8, Granjas')

df_granjas_ingresos = df_granjas[df_granjas[['Unnamed: 1']].dropna().index[0]:
                  df_granjas[df_granjas['Unnamed: 1'] == 'TOTAL INGRESOS'].index[0]]


col_conserved = []
for i, col in enumerate(df_granjas_ingresos.columns):
    if i == 0 and df_granjas_ingresos[col].dropna().empty:
        continue
    elif df_granjas_ingresos[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_granjas_ingresos = df_granjas_ingresos[col_conserved]
cols_strings = ['Producto']
df_granjas_ingresos.columns = cols_strings+cols_range_dates 
df_granjas_ingresos[cols_range_dates] = df_granjas_ingresos[cols_range_dates].fillna(0)
df_granjas_ingresos['Budget'] = df_granjas_ingresos[cols_range_dates].sum(axis = 1)
df_granjas_ingresos['Rubro'] = df_granjas_ingresos['Producto']
df_granjas_ingresos['Partida'] = 'Ingreso granjas OyM'
df_granjas_ingresos['Nodo'] = 'Ingresos'
df_granjas_ingresos['Operación'] = 'I'

# Costo granjas
# ==============================================
df_granjas_costos = df_granjas[df_granjas[df_granjas['Unnamed: 1'] == 'TOTAL INGRESOS'].index[0]+1
                               :df_granjas[df_granjas['Unnamed: 1'] == 'TOTAL GASTOS'].index[0]]


col_conserved = []
for i, col in enumerate(df_granjas_costos.columns):
    if i == 0 and df_granjas_costos[col].dropna().empty:
        continue
    elif df_granjas_costos[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_granjas_costos = df_granjas_costos[col_conserved]
cols_strings = ['Rubro','Producto']
df_granjas_costos.columns = cols_strings+cols_range_dates 
df_granjas_costos[cols_strings] = df_granjas_costos[cols_strings].ffill()
df_granjas_costos[cols_range_dates] = df_granjas_costos[cols_range_dates].fillna(0)
df_granjas_costos['Budget'] = df_granjas_costos[cols_range_dates].sum(axis = 1)
df_granjas_costos['Partida'] = 'Costo granjas OyM'
df_granjas_costos['Nodo'] = 'Costos'
df_granjas_costos['Operación'] = 'E'

df_granjas_concat = pd.concat([df_granjas_ingresos, df_granjas_costos], axis = 0)

In [5]:
# Utilidad bruta
# ==============================================
df_ub = pd.concat([df_epc_concat, df_energia_concat, df_granjas_concat], axis = 0)
df_ub.loc[df_ub['Operación'] == 'E', cols_range_dates] *= -1
df_ub = pd.DataFrame([df_ub[cols_range_dates].sum(axis= 0)])
df_ub['Rubro'] = 'Utilidad bruta'
df_ub['Producto'] = 'Utilidad bruta'
df_ub['Budget'] = df_ub[cols_range_dates].sum(axis = 1)
df_ub['Partida'] = 'Utilidad bruta'
df_ub['Nodo'] = 'Utilidad bruta'
df_ub['Operación'] = 'I'
df_ub_concat = df_ub.copy()

In [ ]:
# Gastos administrativos
# ==============================================
df_gastos_admon = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='1.Adm-Ven-imp')

df_gastos_admon = df_gastos_admon[
                df_gastos_admon[df_gastos_admon['Unnamed: 1'] == 'Arriendo'].index[0]:
                df_gastos_admon[df_gastos_admon['Unnamed: 1'] == 'SALARIOS'].index[0]+1]

col_conserved = []
for i, col in enumerate(df_gastos_admon.columns):
    if i == 0 and df_gastos_admon[col].dropna().empty:
        continue
    elif df_gastos_admon[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_gastos_admon = df_gastos_admon[col_conserved]
cols_strings = ['Producto']
df_gastos_admon.columns = cols_strings+cols_range_dates 
df_gastos_admon[cols_range_dates] = df_gastos_admon[cols_range_dates].fillna(0)
df_gastos_admon['Budget'] = df_gastos_admon[cols_range_dates].sum(axis = 1)
df_gastos_admon['Partida'] = 'Gastos administrativos'
df_gastos_admon['Rubro'] = 'Gastos administrativos'
df_gastos_admon['Nodo'] = 'Gastos'
df_gastos_admon['Operación'] = 'E'

df_gastos_admon_concat = df_gastos_admon.copy()

In [7]:
# Gastos adminitrativos financiero
# ==============================================
df_gastos_admon_finz = pd.DataFrame([[0,0,0,
                                14784910,
                                7264178,
                                7264178,
                                10000000,
                                10000000,
                                10000001,
                                10000001,
                                10000002,
                                10000002]], columns= cols_range_dates)

df_gastos_admon_finz['Budget'] = df_gastos_admon_finz.sum(axis = 1)
df_gastos_admon_finz['Rubro'] = 'Gastos administrativos financiero'
df_gastos_admon_finz['Partida'] = 'Gastos administrativos financiero'
df_gastos_admon_finz['Producto'] = 'Gastos administrativos financiero'
df_gastos_admon_finz['Nodo'] = 'Gastos'
df_gastos_admon_finz['Operación'] = 'E'

df_gastos_admon_finz_concat = df_gastos_admon_finz.copy()


In [8]:
# Gastos intereses
# ==============================================
df_gastos_intereses = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='5. Deuda')
df_gastos_intereses  = df_gastos_intereses[df_gastos_intereses['Unnamed: 1'].notna()]

col_conserved = []
for i, col in enumerate(df_gastos_intereses.columns):
    if i == 0 and df_gastos_intereses[col].dropna().empty:
        continue
    elif df_gastos_intereses[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_gastos_intereses = df_gastos_intereses[col_conserved]
df_gastos_intereses = df_gastos_intereses.drop(columns = ['Intereses'])
cols_strings = ['Producto']
df_gastos_intereses.columns = cols_strings+cols_range_dates 
df_gastos_intereses[cols_range_dates] = df_gastos_intereses[cols_range_dates].fillna(0)
df_gastos_intereses = df_gastos_intereses.reset_index(drop=True)
df_intereses = df_gastos_intereses[
                df_gastos_intereses[(df_gastos_intereses['Producto'] == 'TOTAL')].index[0]+1:
                df_gastos_intereses[(df_gastos_intereses['Producto'] == 'TOTAL')].index[1]]

df_amortizacion = df_gastos_intereses[
                df_gastos_intereses[(df_gastos_intereses['Producto'] == 'TOTAL')].index[1]+1:
                df_gastos_intereses[(df_gastos_intereses['Producto'] == 'TOTAL')].index[2]]

df_saldos = df_gastos_intereses[
                df_gastos_intereses[(df_gastos_intereses['Producto'] == 'TOTAL')].index[2]+1:
                ]

df_intereses['Budget'] = df_intereses[cols_range_dates].sum(axis = 1)
df_intereses['Partida'] = 'Gastos intereses'
df_intereses['Rubro'] = df_intereses['Partida']
df_intereses['Nodo'] = 'Gastos'
df_intereses['Operación'] = 'E'
df_gastos_intereses_concat = df_intereses.copy()

In [9]:
# Gastos ventas
# ==============================================
df_gastos_ventas = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='1.Adm-Ven-imp')

df_gastos_ventas = df_gastos_ventas[
                df_gastos_ventas[df_gastos_ventas['Unnamed: 1'] == 'viaticos comercial '].index[0]:
                df_gastos_ventas[df_gastos_ventas['Unnamed: 1'] == 'Salarios'].index[0]+1]

col_conserved = []
for i, col in enumerate(df_gastos_ventas.columns):
    if i == 0 and df_gastos_ventas[col].dropna().empty:
        continue
    elif df_gastos_ventas[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_gastos_ventas = df_gastos_ventas[col_conserved]
cols_strings = ['Producto']
df_gastos_ventas.columns = cols_strings+cols_range_dates 
df_gastos_ventas[cols_range_dates] = df_gastos_ventas[cols_range_dates].fillna(0)
df_gastos_ventas['Budget'] = df_gastos_ventas[cols_range_dates].sum(axis = 1)
df_gastos_ventas['Partida'] = 'Gastos ventas'
df_gastos_ventas['Nodo'] = 'Gastos'
df_gastos_ventas['Operación'] = 'E'
df_gastos_ventas = df_gastos_ventas[~(df_gastos_ventas['Producto'].isna())].copy()
df_gastos_ventas_concat = df_gastos_ventas.copy()

In [10]:
# Utilidad operativa
# ==============================================
df_uop = pd.concat([df_ub, 
                    df_gastos_admon_concat,
                      df_gastos_admon_finz_concat,
                      df_gastos_intereses_concat,
                      df_gastos_ventas_concat
                      ], axis = 0)
df_uop.loc[df_uop['Operación'] == 'E', cols_range_dates] *= -1
df_uop = pd.DataFrame([df_uop[cols_range_dates].sum(axis= 0)])
df_uop['Rubro'] = 'Utilidad operativa'
df_uop['Producto'] = 'Utilidad operativa'
df_uop['Budget'] = df_uop[cols_range_dates].sum(axis = 1)
df_uop['Partida'] = 'Utilidad operativa'
df_uop['Nodo'] = 'Utilidad operativa'
df_uop['Operación'] = 'I'
df_uop_concat = df_uop.copy()


In [11]:
# Impuestos
# ==============================================
df_impuestos_concat = pd.DataFrame()
for imp_desc in ['Impuestos de Renta',
                'Impuesto ICA',
                'Beneficio de Renta'
                ]:
    
    df_temp = pd.DataFrame([[0,0,0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0]], columns= cols_range_dates)

    df_temp['Budget'] = df_temp.sum(axis = 1)
    df_temp['Rubro'] = imp_desc
    df_temp['Partida'] = imp_desc
    df_temp['Producto'] = imp_desc
    df_temp['Nodo'] = 'Impuestos'
    df_temp['Operación'] = 'E'

    df_impuestos_concat = pd.concat([df_impuestos_concat, df_temp])

In [12]:
# NOPLAT
# ==============================================
df_noplat = pd.concat([df_uop, 
                    df_impuestos_concat
                      ], axis = 0)
df_noplat.loc[df_noplat['Operación'] == 'E', cols_range_dates] *= -1
df_noplat = pd.DataFrame([df_noplat[cols_range_dates].sum(axis= 0)])
df_noplat['Rubro'] = 'NOPLAT'
df_noplat['Producto'] = 'NOPLAT'
df_noplat['Budget'] = df_noplat[cols_range_dates].sum(axis = 1)
df_noplat['Partida'] = 'NOPLAT'
df_noplat['Nodo'] = 'NOPLAT'
df_noplat['Operación'] = 'I'
df_noplat_concat = df_noplat.copy()


In [13]:
# Depreciación + Amortización
# ==============================================
df_depamor_concat = pd.DataFrame()
for imp_desc in ['Depreciación & Amortización']:
    
    df_temp = pd.DataFrame([[0,0,0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0]], columns= cols_range_dates)

    df_temp['Budget'] = df_temp.sum(axis = 1)
    df_temp['Rubro'] = imp_desc
    df_temp['Partida'] = imp_desc
    df_temp['Producto'] = imp_desc
    df_temp['Nodo'] = imp_desc
    df_temp['Operación'] = 'E'

    df_depamor_concat = pd.concat([df_depamor_concat, df_temp])

In [14]:
# Flujo de caja operacional
# ==============================================
df_fco = pd.concat([df_noplat, 
                    df_depamor_concat
                      ], axis = 0)
df_fco.loc[df_fco['Operación'] == 'E', cols_range_dates] *= -1
df_fco = pd.DataFrame([df_fco[cols_range_dates].sum(axis= 0)])
df_fco['Rubro'] = 'Flujo de caja operacional'
df_fco['Producto'] = 'Flujo de caja operacional'
df_fco['Budget'] = df_fco[cols_range_dates].sum(axis = 1)
df_fco['Partida'] = 'Flujo de caja operacional'
df_fco['Nodo'] = 'Flujo de caja operacional'
df_fco['Operación'] = 'I'
df_fco_concat = df_fco.copy()


In [15]:
# Inversión granjas
# ==============================================
df_inversion_granjas = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='Inversión granjas')
col_conserved = []

for i, col in enumerate(df_inversion_granjas.columns):
    if i == 0 and df_inversion_granjas[col].dropna().empty:
        continue
    elif df_inversion_granjas[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_inversion_granjas = df_inversion_granjas[col_conserved]
df_inversion_granjas = df_inversion_granjas[df_inversion_granjas[['Unnamed: 1']].dropna().index[0]:df_inversion_granjas[['Unnamed: 4']].dropna().index[-1]][col_conserved]
cols_strings = ['Producto','Partida']
df_inversion_granjas.columns = cols_strings+cols_range_dates 
df_inversion_granjas[cols_strings] = df_inversion_granjas[cols_strings].ffill()
df_inversion_granjas[cols_range_dates] = df_inversion_granjas[cols_range_dates].fillna(0)
df_inversion_granjas['Nodo'] = 'Inversiones'
df_inversion_granjas['Rubro'] = 'Inversiones Evolti'
df_inversion_granjas['Budget'] = df_inversion_granjas[cols_range_dates].sum(axis = 1)
df_inversion_granjas['Operación'] = 'I'
df_inversion_granjas_concat = df_inversion_granjas[df_inversion_granjas['Partida'] == 'Total']
df_inversion_granjas_concat['Partida'] = 'Δ Inversión granjas'

In [16]:
# Inversión Evolti propio
# ==============================================
df_inversion_evolti_propio = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='4 EPC - MINI GRANJAS - CAPEX')
df_inversion_evolti_propio = df_inversion_evolti_propio.drop(columns=df_inversion_evolti_propio.columns.to_list()[:2])
col_conserved = []

for i, col in enumerate(df_inversion_evolti_propio.columns):
    if i == 0 and df_inversion_evolti_propio[col].dropna().empty:
        continue
    elif df_inversion_evolti_propio[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_inversion_evolti_propio = df_inversion_evolti_propio[col_conserved]
cols_strings = ['Unnamed: 2','Producto','Partida']
df_inversion_evolti_propio.columns = cols_strings+cols_range_dates 
df_inversion_evolti_propio[cols_strings] = df_inversion_evolti_propio[cols_strings].ffill()
df_inversion_evolti_propio[cols_range_dates] = df_inversion_evolti_propio[cols_range_dates].fillna(0)
df_inversion_evolti_propio = df_inversion_evolti_propio[df_inversion_evolti_propio['Unnamed: 2'] == 'MG'].drop(columns= 'Unnamed: 2')
df_inversion_evolti_propio = df_inversion_evolti_propio[df_inversion_evolti_propio['Producto'] == 'EVOLTI GENERACIÓN']
df_inversion_evolti_propio = df_inversion_evolti_propio[df_inversion_evolti_propio['Partida'] == 'Total']
df_inversion_evolti_propio['Nodo'] = 'Inversiones'
df_inversion_evolti_propio['Rubro'] = 'Inversiones Evolti'
df_inversion_evolti_propio['Budget'] = df_inversion_evolti_propio[cols_range_dates].sum(axis = 1)
df_inversion_evolti_propio['Operación'] = 'I'
df_inversion_evolti_propio_concat = df_inversion_evolti_propio[df_inversion_evolti_propio['Partida'] == 'Total']
df_inversion_evolti_propio_concat['Partida'] = 'Inversión Evolti'


In [17]:
# CAPEX PPAs
# ==============================================
df_capex_ppa = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='4 EPC - MINI GRANJAS - CAPEX')
df_capex_ppa = df_capex_ppa.drop(columns=df_capex_ppa.columns.to_list()[:2])
col_conserved = []

for i, col in enumerate(df_capex_ppa.columns):
    if i == 0 and df_capex_ppa[col].dropna().empty:
        continue
    elif df_capex_ppa[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_capex_ppa = df_capex_ppa[col_conserved]
cols_strings = ['Unnamed: 2','Producto','Partida']
df_capex_ppa.columns = cols_strings+cols_range_dates 
df_capex_ppa['Unnamed: 2'] = df_capex_ppa['Unnamed: 2'].ffill()
df_capex_ppa = df_capex_ppa[df_capex_ppa['Unnamed: 2'] == 'PPA']
df_capex_ppa = df_capex_ppa[df_capex_ppa['Partida'].notna()]
df_capex_ppa[cols_strings] = df_capex_ppa[cols_strings].ffill()
df_capex_ppa[cols_range_dates] = df_capex_ppa[cols_range_dates].fillna(0)
df_capex_ppa = df_capex_ppa[df_capex_ppa['Partida'] == 'Total'].drop(columns= 'Unnamed: 2')
df_capex_ppa['Nodo'] = 'Inversiones'
df_capex_ppa['Rubro'] = 'Inversiones Evolti'
df_capex_ppa['Budget'] = df_capex_ppa[cols_range_dates].sum(axis = 1)
df_capex_ppa['Operación'] = 'I'
df_capex_ppa = df_capex_ppa[df_capex_ppa['Partida'] == 'Total']
df_capex_ppa['Partida'] = 'CAPEX PPAs'
df_capex_ppa_concat = df_capex_ppa.copy()


In [18]:
# EPC
# ==============================================
df_epc_inversiones = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='4 EPC - MINI GRANJAS - CAPEX')
df_epc_inversiones = df_epc_inversiones.drop(columns=df_epc_inversiones.columns.to_list()[:2])
col_conserved = []

for i, col in enumerate(df_epc_inversiones.columns):
    if i == 0 and df_epc_inversiones[col].dropna().empty:
        continue
    elif df_epc_inversiones[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_epc_inversiones = df_epc_inversiones[col_conserved]
cols_strings = ['Unnamed: 2','Producto','Partida']
df_epc_inversiones.columns = cols_strings+cols_range_dates 
df_epc_inversiones['Unnamed: 2'] = df_epc_inversiones['Unnamed: 2'].ffill()
df_epc_inversiones = df_epc_inversiones[df_epc_inversiones['Unnamed: 2'] == 'EPC']
df_epc_inversiones = df_epc_inversiones[df_epc_inversiones['Partida'].notna()]
df_epc_inversiones[cols_strings] = df_epc_inversiones[cols_strings].ffill()
df_epc_inversiones[cols_range_dates] = df_epc_inversiones[cols_range_dates].fillna(0)
df_epc_inversiones = df_epc_inversiones[df_epc_inversiones['Partida'] == 'Total'].drop(columns= 'Unnamed: 2')
df_epc_inversiones['Nodo'] = 'KT'
df_epc_inversiones['Rubro'] = 'Techos'
df_epc_inversiones['Budget'] = df_epc_inversiones[cols_range_dates].sum(axis = 1)
df_epc_inversiones['Operación'] = 'I'
df_epc_inversiones = df_epc_inversiones[df_epc_inversiones['Partida'] == 'Total']
df_epc_inversiones['Partida'] = 'EPC'
df_epc_inversiones_concat = df_epc_inversiones.copy()


In [19]:
# Minigranjas Minigranjas
# ==============================================
df_inversion_minigranjas = pd.read_excel("./data/Planeacion 2025 Flujo de Caja.xlsx", 
                              sheet_name='4 EPC - MINI GRANJAS - CAPEX')
df_inversion_minigranjas = df_inversion_minigranjas.drop(columns=df_inversion_minigranjas.columns.to_list()[:2])
col_conserved = []

for i, col in enumerate(df_inversion_minigranjas.columns):
    if i == 0 and df_inversion_minigranjas[col].dropna().empty:
        continue
    elif df_inversion_minigranjas[col].dropna().empty:
        break
    else:
        col_conserved.append(col)

df_inversion_minigranjas = df_inversion_minigranjas[col_conserved]
cols_strings = ['Unnamed: 2','Producto','Partida']
df_inversion_minigranjas.columns = cols_strings+cols_range_dates 
df_inversion_minigranjas[cols_strings] = df_inversion_minigranjas[cols_strings].ffill()
df_inversion_minigranjas[cols_range_dates] = df_inversion_minigranjas[cols_range_dates].fillna(0)
df_inversion_minigranjas = df_inversion_minigranjas[df_inversion_minigranjas['Unnamed: 2'] == 'MG'].drop(columns= 'Unnamed: 2')
df_inversion_minigranjas = df_inversion_minigranjas[df_inversion_minigranjas['Producto'] != 'EVOLTI GENERACIÓN']
df_inversion_minigranjas = df_inversion_minigranjas[df_inversion_minigranjas['Partida'] == 'Total']
df_inversion_minigranjas['Nodo'] = 'KT'
df_inversion_minigranjas['Rubro'] = 'Minigranjas'
df_inversion_minigranjas['Budget'] = df_inversion_minigranjas[cols_range_dates].sum(axis = 1)
df_inversion_minigranjas['Operación'] = 'I'
df_inversion_minigranjas_concat = df_inversion_minigranjas[df_inversion_minigranjas['Partida'] == 'Total']
df_inversion_minigranjas_concat['Partida'] = 'Minigranjas'


In [20]:
# Minigranjas Estructuración
# ==============================================
df_minigranjas_estructuracion_concat = pd.DataFrame()
for _desc in ['Estructuración']:
    
    df_temp = pd.DataFrame([[0,0,0,
                                    -82450356,
                                    -81144854,
                                    -69000000,
                                    -69000000,
                                    -69000000,
                                    -69000000,
                                    -69000000,
                                    -69000000,
                                    -69000000]], columns= cols_range_dates)

    df_temp['Budget'] = df_temp.sum(axis = 1)
    df_temp['Rubro'] = 'Minigranjas'
    df_temp['Partida'] = _desc
    df_temp['Producto'] = _desc
    df_temp['Nodo'] = 'KT'
    df_temp['Operación'] = 'I'

    df_minigranjas_estructuracion_concat = pd.concat([df_minigranjas_estructuracion_concat, df_temp])


In [21]:
# Devolución inventario / impuestos 
# ==============================================
df_devolucion_inventario = df_epc_concat.copy()
df_devolucion_inventario.loc[df_devolucion_inventario['Operación'] == 'I', cols_range_dates] *= -1
df_devolucion_inventario = pd.DataFrame([df_devolucion_inventario[cols_range_dates].sum(axis= 0)])
df_devolucion_inventario['Rubro'] = 'Devolución inventario / impuestos'
df_devolucion_inventario['Producto'] = 'Devolución inventario / impuestos'
df_devolucion_inventario['Budget'] = df_devolucion_inventario[cols_range_dates].sum(axis = 1)
df_devolucion_inventario['Partida'] = 'Devolución inventario / impuestos'
df_devolucion_inventario['Nodo'] = 'KT'
df_devolucion_inventario['Operación'] = 'I'
df_devolucion_inventario_concat = df_devolucion_inventario.copy()


In [22]:
# df_devolucion_inventario[['2025-12']]
df_devolucion_inventario = df_epc_concat.copy()
df_devolucion_inventario.loc[df_devolucion_inventario['Operación'] == 'I', cols_range_dates] *= -1
df_devolucion_inventario = pd.DataFrame([df_devolucion_inventario[cols_range_dates].sum(axis= 0)])
df_devolucion_inventario

,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12
0,0.0,-1.841597e+09,-32103000.0,-3.467450e+08,-707405180.5,-194898647.1,-67393291.4,-982008278.5,-804000000.0,-52060000.0,-1.636120e+09,-4.718796e+09


In [23]:
# Capital de trabajo
# ==============================================
df_capital_trabajo = pd.concat([
                    df_epc_inversiones_concat,
                    df_inversion_minigranjas_concat,
                    df_minigranjas_estructuracion_concat,
                    df_devolucion_inventario_concat
                      ], axis = 0)
df_capital_trabajo.loc[df_capital_trabajo['Operación'] == 'E', cols_range_dates] *= -1
df_capital_trabajo = pd.DataFrame([df_capital_trabajo[cols_range_dates].sum(axis= 0)])
df_capital_trabajo['Rubro'] = 'Capital de trabajo'
df_capital_trabajo['Producto'] = 'Δ Capital de trabajo'
df_capital_trabajo['Budget'] = df_capital_trabajo[cols_range_dates].sum(axis = 1)
df_capital_trabajo['Partida'] = 'Δ Capital de trabajo'
df_capital_trabajo['Nodo'] = 'Capital de trabajo'
df_capital_trabajo['Operación'] = 'I'
df_capital_trabajo_concat = df_capital_trabajo.copy()

In [24]:
# Flujo de caja libre
# ==============================================
df_fcl = pd.concat([df_inversion_granjas_concat,
                    df_inversion_evolti_propio_concat,
                    df_capex_ppa_concat,
                    df_epc_inversiones_concat,
                    df_inversion_minigranjas_concat,
                    df_minigranjas_estructuracion_concat,
                    df_devolucion_inventario_concat
                      ], axis = 0)

df_fcl.loc[df_fcl['Operación'] == 'E', cols_range_dates] *= -1
df_fcl = pd.DataFrame([df_fcl[cols_range_dates].sum(axis= 0)])
df_fcl['Rubro'] = 'Flujo de caja libre'
df_fcl['Producto'] = 'Flujo de caja libre'
df_fcl['Budget'] = df_fcl[cols_range_dates].sum(axis = 1)
df_fcl['Partida'] = 'Flujo de caja libre'
df_fcl['Nodo'] = 'Flujo de caja libre'
df_fcl['Operación'] = 'I'
df_fcl_concat = df_fcl.copy()


In [25]:
# Deuda Actual
# ==============================================
df_amortizacion['Budget'] = df_amortizacion[cols_range_dates].sum(axis = 1)
df_amortizacion['Partida'] = 'Δ Deuda Actual'
df_amortizacion['Rubro'] = df_amortizacion['Partida']
df_amortizacion['Nodo'] = 'Accionistas'
df_amortizacion[cols_range_dates] *= -1
df_amortizacion['Operación'] = 'I'
df_amortizacion_concat = df_amortizacion.copy()

In [26]:
# Δ Deuda Para PPAs
# ==============================================

df_deuda_ppas = pd.DataFrame([[0,0,0,
                                0,
                                0,
                                0,
                                0,
                                0,
                                0,
                                0,
                                0,
                                0]], columns= cols_range_dates)

df_deuda_ppas['Budget'] = df_deuda_ppas.sum(axis = 1)
df_deuda_ppas['Rubro'] = 'Deuda Para PPAs'
df_deuda_ppas['Partida'] = 'Δ Deuda Para PPAs'
df_deuda_ppas['Producto'] = 'Δ Deuda Para PPAs'
df_deuda_ppas['Nodo'] = 'Accionistas'
df_deuda_ppas['Operación'] = 'I'
df_deuda_ppas_concat = df_deuda_ppas.copy()

In [27]:
# Flujo de caja del accionista
# ==============================================
df_fca = pd.concat([df_fcl_concat,
                    df_amortizacion_concat,
                    df_deuda_ppas_concat,
                      ], axis = 0)

df_fca.loc[df_fca['Operación'] == 'E', cols_range_dates] *= -1
df_fca = pd.DataFrame([df_fca[cols_range_dates].sum(axis= 0)])
df_fca['Rubro'] = 'Flujo de caja del accionista'
df_fca['Producto'] = 'Flujo de caja del accionista'
df_fca['Budget'] = df_fca[cols_range_dates].sum(axis = 1)
df_fca['Partida'] = 'Flujo de caja del accionista'
df_fca['Nodo'] = 'Flujo de caja del accionista'
df_fca['Operación'] = 'I'
df_fca_concat = df_fca.copy()


In [28]:
# Resultado del ejercicio
# ==============================================
df_pat_concat = pd.DataFrame()
for _desc in ['Δ Capital y superhavit de capital' ,  '- Dividendos']:
    
    df_temp = pd.DataFrame([[0,0,0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0,
                                    0]], columns= cols_range_dates)

    df_temp['Budget'] = df_temp.sum(axis = 1)
    df_temp['Rubro'] = _desc
    df_temp['Partida'] = _desc
    df_temp['Producto'] = _desc
    df_temp['Nodo'] = 'Resultado_Ejercicio'
    df_temp['Operación'] = 'I'

    df_pat_concat = pd.concat([df_pat_concat, df_temp])

In [29]:
# Flujo de caja neto
# ==============================================
df_fcn = pd.concat([df_fca_concat,
                    df_pat_concat,
                      ], axis = 0)

df_fcn.loc[df_fcn['Operación'] == 'E', cols_range_dates] *= -1
df_fcn = pd.DataFrame([df_fcn[cols_range_dates].sum(axis= 0)])
df_fcn['Rubro'] = 'Flujo de caja neto'
df_fcn['Producto'] = 'Flujo de caja neto'
df_fcn['Budget'] = df_fcn[cols_range_dates].sum(axis = 1)
df_fcn['Partida'] = 'Flujo de caja neto'
df_fcn['Nodo'] = 'Flujo de caja neto'
df_fcn['Operación'] = 'I'
df_fcn_concat = df_fcn.copy()

In [30]:
# Caja inicial
# ==============================================


df_caja_final = pd.DataFrame()
for num, desc_date in enumerate(cols_range_dates):
    if num == 0:
        df_caja_inicial = pd.DataFrame([[228808924]], columns= [desc_date])
        df_caja_final[desc_date] = pd.concat([df_fcn_concat[[desc_date]], df_caja_inicial], axis = 0).sum().values
    else:
        df_caja_inicial[desc_date] = df_caja_final[cols_range_dates[num-1]]
        df_caja_final[desc_date] = pd.concat([df_fcn_concat[[desc_date]], df_caja_inicial[[desc_date]]], axis = 0).sum().values


df_caja_inicial['Rubro'] = 'Caja inicial'
df_caja_inicial['Producto'] = 'Caja inicial'
df_caja_inicial['Budget'] = df_caja_inicial[cols_range_dates].sum(axis = 1)
df_caja_inicial['Partida'] = 'Caja inicial'
df_caja_inicial['Nodo'] = 'Caja inicial'
df_caja_inicial['Operación'] = 'I'
df_caja_inicial_concat = df_caja_inicial.copy()

df_caja_final['Rubro'] = 'Caja final'
df_caja_final['Producto'] = 'Caja final'
df_caja_final['Budget'] = df_caja_final[cols_range_dates].sum(axis = 1)
df_caja_final['Partida'] = 'Caja final'
df_caja_final['Nodo'] = 'Caja final'
df_caja_final['Operación'] = 'I'
df_caja_final_concat = df_caja_final.copy()

In [206]:
df_concat = pd.DataFrame()
for num, frame in enumerate([df_epc_concat
    , df_energia_concat
    , df_granjas_concat
    , df_ub_concat
    , df_gastos_admon_concat
    , df_gastos_admon_finz_concat
    , df_gastos_intereses_concat
    , df_gastos_ventas_concat
    , df_uop_concat
    , df_impuestos_concat
    , df_noplat_concat
    , df_depamor_concat
    , df_fco_concat
    , df_inversion_granjas_concat
    , df_inversion_evolti_propio_concat
    , df_capex_ppa_concat
    , df_epc_inversiones_concat
    , df_inversion_minigranjas_concat
    , df_minigranjas_estructuracion_concat
    , df_devolucion_inventario_concat
    , df_capital_trabajo_concat
    , df_fcl_concat
    , df_amortizacion_concat
    , df_deuda_ppas_concat
    , df_fca_concat
    , df_pat_concat
    , df_fcn_concat
    , df_caja_inicial_concat
    , df_caja_final_concat]):
        df_temp = frame.copy()
        df_temp['Orden_Partida'] = num
        df_concat = pd.concat([df_concat, df_temp], axis = 0)

for num, node in enumerate(df_concat['Nodo'].unique().tolist()):
    df_concat.loc[df_concat['Nodo'] == node, 'Orden_Nodo'] = num


df_concat = df_concat.drop(columns = ['Budget','Operación'])

In [207]:
df_concat = df_concat.melt(
    id_vars=['Rubro', 'Producto', 'Partida', 'Nodo', 'Orden_Partida', 'Orden_Nodo'],
    value_vars=cols_range_dates,
    var_name='Fecha',
    value_name='Budget'
)
df_concat = df_concat[['Fecha','Nodo','Partida','Rubro','Producto','Orden_Partida','Orden_Nodo','Budget']]
df_concat['Statemens'] = 'Budget'

df_concat.to_csv("./data/df_concat.csv", index=False)

# Real



In [208]:
# Ventas EPC
# ==============================================
df_mvtos_reales = pd.read_excel("./data/Administrativo 2025.xlsx", 
                            sheet_name='Movimientos Reales')

col_conserved = []

for i, col in enumerate(df_mvtos_reales.columns):
    if i == 0 and df_mvtos_reales[col].dropna().empty:
        continue
    elif df_mvtos_reales[col].dropna().empty:
        break
    else:
        col_conserved.append(col)
col_conserved = col_conserved[:-1]
df_mvtos_reales = df_mvtos_reales[df_mvtos_reales[['Unnamed: 0']].dropna().index[0]:][col_conserved].reset_index(drop=True)
df_mvtos_reales.columns = df_mvtos_reales.iloc[0]
df_mvtos_reales = df_mvtos_reales[df_mvtos_reales['Fecha'].index[0]+1:]
df_mvtos_reales['Monto'] = df_mvtos_reales['Monto'].apply(lambda x: x if isinstance(x, (int, float)) else 0)
for c in ['Apellido','Apellido2', 'Nombre']:
    df_mvtos_reales[c] = df_mvtos_reales[c].str.lower()

df_mvtos_reales = df_mvtos_reales.rename(columns ={'Concepto ':'Producto', 'Monto':'Real'})
df_mvtos_reales['Fecha'] = pd.to_datetime(df_mvtos_reales['MES'], format='%Y-%m').dt.strftime('%Y-%m')
df_mvtos_reales['Producto'] = np.where(df_mvtos_reales['Apellido2'].isnull(),
                                    df_mvtos_reales['Apellido'],
                                    df_mvtos_reales['Apellido2'])

In [209]:
# Ingresos reales
# ==============================================
dict_ingresos_reales = {
'gd':['Ingresos','Ventas EPC','Mini granjas'],
'ingresos gd':['Ingresos','Ventas EPC','Mini granjas'],
'gen':['Ingresos','Ingreso por venta de energía','Energía'],
}
dict_to_work = dict_ingresos_reales.copy()
df_ingresos_real = df_mvtos_reales.copy()
df_ingresos_real['Rubro'] = df_ingresos_real['Nombre'].map(dict_to_work).apply(lambda x: x[2] if isinstance(x, list) else x)
df_ingresos_real['Partida'] = df_ingresos_real['Nombre'].map(dict_to_work).apply(lambda x: x[1] if isinstance(x, list) else x)
df_ingresos_real['Nodo'] = df_ingresos_real['Nombre'].map(dict_to_work).apply(lambda x: x[0] if isinstance(x, list) else x)
df_ingresos_real['Producto'] = np.where(df_ingresos_real['Producto'].isnull(),
                                    df_ingresos_real['Partida'],
                                    df_ingresos_real['Producto'])

df_ingresos_real = df_ingresos_real.merge(df_concat[['Nodo','Orden_Nodo']].drop_duplicates(), on='Nodo', how='left')
df_ingresos_real = df_ingresos_real.merge(df_concat[['Partida','Orden_Partida']].drop_duplicates(), on='Partida', how='left')
df_ingresos_real = df_ingresos_real[['Fecha','Nodo','Partida','Rubro','Producto','Orden_Partida','Orden_Nodo','Real']]
df_ingresos_real = df_ingresos_real[~(df_ingresos_real['Nodo'].isnull())]

In [210]:
# Costos reales
# ==============================================
dict_costos_reales = {
'mantenimiento':['Costos','Costos por venta de energía','Energía'],
'costos mini granja':['Costos','Costos EPC','Mini granjas'],
}
dict_to_work = dict_costos_reales.copy()
df_costos_real = df_mvtos_reales.copy()
df_costos_real['Rubro'] = df_costos_real['Nombre'].map(dict_to_work).apply(lambda x: x[2] if isinstance(x, list) else x)
df_costos_real['Partida'] = df_costos_real['Nombre'].map(dict_to_work).apply(lambda x: x[1] if isinstance(x, list) else x)
df_costos_real['Nodo'] = df_costos_real['Nombre'].map(dict_to_work).apply(lambda x: x[0] if isinstance(x, list) else x)
df_costos_real['Producto'] = np.where(df_costos_real['Producto'].isnull(),
                                    df_costos_real['Partida'],
                                    df_costos_real['Producto'])

df_costos_real = df_costos_real.merge(df_concat[['Nodo','Orden_Nodo']].drop_duplicates(), on='Nodo', how='left')
df_costos_real = df_costos_real.merge(df_concat[['Partida','Orden_Partida']].drop_duplicates(), on='Partida', how='left')
df_costos_real = df_costos_real[['Fecha','Nodo','Partida','Rubro','Producto','Orden_Partida','Orden_Nodo','Real']]
df_costos_real = df_costos_real[~(df_costos_real['Nodo'].isnull())]

df_costos_real_plus = df_costos_real.copy()
df_costos_real_plus['Real'] = df_costos_real_plus['Real']*-1

In [211]:
# Utilidad bruta
# ==============================================
df_ub_real = pd.concat([df_ingresos_real, df_costos_real], axis = 0)
df_ub_real = df_ub_real.groupby(['Fecha'])['Real'].sum().reset_index()
df_ub_real['Rubro'] = 'Utilidad bruta'
df_ub_real['Producto'] = 'Utilidad bruta'
df_ub_real['Partida'] = 'Utilidad bruta'
df_ub_real['Nodo'] = 'Utilidad bruta'
df_ub_real = df_ub_real.merge(df_concat[['Nodo','Orden_Nodo']].drop_duplicates(), on='Nodo', how='left')
df_ub_real = df_ub_real.merge(df_concat[['Partida','Orden_Partida']].drop_duplicates(), on='Partida', how='left')

In [212]:
# Gastos reales
# ==============================================
dict_gasto_reales = {
'admon':['Gastos','Gastos administrativos','Gastos administrativos'],
'intereses':['Gastos','Gastos intereses','Gastos intereses'],
}
dict_to_work = dict_gasto_reales.copy()
df_gasto_real = df_mvtos_reales.copy()
df_gasto_real['Rubro'] = df_gasto_real['Nombre'].map(dict_to_work).apply(lambda x: x[2] if isinstance(x, list) else x)
df_gasto_real['Partida'] = df_gasto_real['Nombre'].map(dict_to_work).apply(lambda x: x[1] if isinstance(x, list) else x)
df_gasto_real['Nodo'] = df_gasto_real['Nombre'].map(dict_to_work).apply(lambda x: x[0] if isinstance(x, list) else x)
df_gasto_real['Producto'] = np.where(df_gasto_real['Producto'].isnull(),
                                    df_gasto_real['Partida'],
                                    df_gasto_real['Producto'])

df_gasto_real = df_gasto_real.merge(df_concat[['Nodo','Orden_Nodo']].drop_duplicates(), on='Nodo', how='left')
df_gasto_real = df_gasto_real.merge(df_concat[['Partida','Orden_Partida']].drop_duplicates(), on='Partida', how='left')

df_gasto_ventas_real = df_gasto_real[df_gasto_real['Apellido'] == 'ventas']
df_gasto_ventas_real['Rubro'] = 'Gastos ventas'
df_gasto_ventas_real['Partida'] = 'Gastos ventas'
df_gasto_real = df_gasto_real[df_gasto_real['Apellido'] != 'ventas']
df_gasto_real = pd.concat([df_gasto_real, df_gasto_ventas_real], axis = 0)
df_gasto_real = df_gasto_real[['Fecha','Nodo','Partida','Rubro','Producto','Orden_Partida','Orden_Nodo','Real']]
df_gasto_real = df_gasto_real[~(df_gasto_real['Nodo'].isnull())]

df_gasto_real_plus = df_gasto_real.copy()
df_gasto_real_plus['Real'] = df_gasto_real_plus['Real']*-1

In [213]:
# Gastos reales
# ==============================================
dict_gasto_reales = {
'admon':['Gastos','Gastos administrativos','Gastos administrativos'],
'intereses':['Gastos','Gastos intereses','Gastos intereses'],
}
dict_to_work = dict_gasto_reales.copy()
df_gasto_real = df_mvtos_reales.copy()
df_gasto_real['Rubro'] = df_gasto_real['Nombre'].map(dict_to_work).apply(lambda x: x[2] if isinstance(x, list) else x)
df_gasto_real['Partida'] = df_gasto_real['Nombre'].map(dict_to_work).apply(lambda x: x[1] if isinstance(x, list) else x)
df_gasto_real['Nodo'] = df_gasto_real['Nombre'].map(dict_to_work).apply(lambda x: x[0] if isinstance(x, list) else x)
df_gasto_real['Producto'] = np.where(df_gasto_real['Producto'].isnull(),
                                    df_gasto_real['Partida'],
                                    df_gasto_real['Producto'])

df_gasto_real = df_gasto_real.merge(df_concat[['Nodo','Orden_Nodo']].drop_duplicates(), on='Nodo', how='left')
df_gasto_real = df_gasto_real.merge(df_concat[['Partida','Orden_Partida']].drop_duplicates(), on='Partida', how='left')

df_gasto_ventas_real = df_gasto_real[df_gasto_real['Apellido'] == 'ventas']
df_gasto_ventas_real['Rubro'] = 'Gastos ventas'
df_gasto_ventas_real['Partida'] = 'Gastos ventas'
df_gasto_real = df_gasto_real[df_gasto_real['Apellido'] != 'ventas']
df_gasto_real = pd.concat([df_gasto_real, df_gasto_ventas_real], axis = 0)
df_gasto_real = df_gasto_real[['Fecha','Nodo','Partida','Rubro','Producto','Orden_Partida','Orden_Nodo','Real']]
df_gasto_real = df_gasto_real[~(df_gasto_real['Nodo'].isnull())]

df_gasto_real_plus = df_gasto_real.copy()
df_gasto_real_plus['Real'] = df_gasto_real_plus['Real']*-1

In [214]:
# Gastos financieros reales
# ==============================================
df_gastos_financieros = pd.read_excel("./data/Administrativo 2025.xlsx", 
                            sheet_name='Gastos Bancarios')

col_conserved = []

for i, col in enumerate(df_gastos_financieros.columns):
    if i == 0 and df_gastos_financieros[col].head(20).dropna().empty:
        continue
    elif df_gastos_financieros[col].head(20).dropna().empty:
        break
    else:
        col_conserved.append(col)
col_conserved = col_conserved[:-1]
df_gastos_financieros = df_gastos_financieros[df_gastos_financieros[[col_conserved[0]]].dropna().index[0]:][col_conserved].reset_index(drop=True)
df_gastos_financieros.columns = df_gastos_financieros.iloc[1]
df_gastos_financieros = df_gastos_financieros[df_gastos_financieros['Fecha'].index[1]+1:]
df_gastos_financieros['Monto'] = df_gastos_financieros['Monto'].apply(lambda x: x if isinstance(x, (int, float)) else 0)
df_gastos_financieros = df_gastos_financieros.dropna()
for c in ['Concepto ']:
    df_gastos_financieros[c] = df_gastos_financieros[c].str.lower()

df_gastos_financieros = df_gastos_financieros.rename(columns ={'Concepto ':'Producto', 'Monto':'Real'})
df_gastos_financieros['Fecha'] = pd.to_datetime(df_gastos_financieros['mes'], format='%Y-%m').dt.strftime('%Y-%m')
df_gastos_financieros['Rubro'] = 'Gastos administrativos financiero'
df_gastos_financieros['Partida'] = 'Gastos administrativos financiero'
df_gastos_financieros['Nodo'] = 'Gastos'

df_gastos_financieros = df_gastos_financieros.merge(df_concat[['Nodo','Orden_Nodo']].drop_duplicates(), on='Nodo', how='left')
df_gastos_financieros = df_gastos_financieros.merge(df_concat[['Partida','Orden_Partida']].drop_duplicates(), on='Partida', how='left')
df_gastos_financieros = df_gastos_financieros[['Fecha','Nodo','Partida','Rubro','Producto','Orden_Partida','Orden_Nodo','Real']]
df_gastos_financieros_plus = df_gastos_financieros.copy()
df_gastos_financieros_plus['Real'] = df_gastos_financieros_plus['Real']*-1

In [215]:
df_concat_real = pd.concat([df_ingresos_real,
                            df_costos_real_plus,
                            df_ub_real,
                            df_gasto_real_plus,
                            df_gastos_financieros_plus], axis=0)
df_concat_real['Statemens'] = 'Real'
df_concat = pd.concat([df_concat,df_concat_real], axis=0)

df_concat.fillna(0, inplace=True)


# Diff Nominal

In [216]:
df_concat_diff = df_concat.copy()
df_concat_diff = df_concat_diff.groupby(['Fecha', 'Nodo', 'Partida', 'Rubro', 'Orden_Partida', 'Orden_Nodo'])[['Budget','Real']].sum().reset_index()
df_concat_diff['diff'] = df_concat_diff['Budget'] - df_concat_diff['Real']
df_concat_diff['Statemens'] = 'Diff'
df_concat_diff['Producto'] = 'Diff'
df_concat_diff = df_concat_diff.drop(columns=['Budget', 'Real'])

# Diff %

In [217]:
df_concat_cumplimiento = df_concat.copy()
df_concat_cumplimiento = df_concat_cumplimiento.groupby(['Fecha', 'Nodo', 'Partida', 'Rubro', 'Orden_Partida', 'Orden_Nodo'])[['Budget','Real']].sum().reset_index()
df_concat_cumplimiento['%'] = np.where(
    df_concat_cumplimiento['Budget'] == 0,
    0,
    (df_concat_cumplimiento['Real'] / df_concat_cumplimiento['Budget'])*100
)

df_concat_cumplimiento['Statemens'] = '%'
df_concat_cumplimiento['Producto'] = '%'
df_concat_cumplimiento = df_concat_cumplimiento.drop(columns=['Budget', 'Real'])

# Union all Df

In [218]:
df_concat = pd.concat([df_concat,df_concat_diff, df_concat_cumplimiento], axis=0)
df_concat.fillna(0, inplace=True)

df_concat['Monto'] = df_concat['Budget'] + df_concat['Real'] + df_concat['diff'] + df_concat['%']

order_statements = {'Budget':0, 'Real':1, 'Diff':2, '%':3}
df_concat['Orden_Statement'] = df_concat['Statemens'].map(order_statements)

df_concat.to_csv("./data/df_concat.csv", index=False)
